In [12]:
!pip install keras-tuner --upgrade

Note: you may need to restart the kernel to use updated packages.


## Primo tokenizer e modello usato
label_map = {label: idx for idx, label in enumerate(y.unique())}
y = y.map(label_map)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
print(len(label_map))

tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)


max_len = 115 # lunghezza massima delle sequenze
X_train_pad = pad_sequences(X_train_seq, maxlen=max_len, padding='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len, padding='post')



model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=100))
model.add(Conv1D(128, 5, activation='relu'))
model.add(Bidirectional(LSTM(64, dropout=0.3, recurrent_dropout=0.5))) # Rimuovi GlobalMaxPooling1D
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(len(label_map), activation='softmax'))



model.compile(loss='sparse_categorical_crossentropy', optimizer=tf.optimizers.Adam(learning_rate=0.001), metrics=['accuracy'])
  

history = model.fit(X_train_pad, y_train, epochs=50, batch_size=64, validation_data=(X_test_pad, y_test))


loss, accuracy = model.evaluate(X_test_pad, y_test)
print("Test Accuracy:", accuracy)


tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['text'])
sequences = tokenizer.texts_to_sequences(df['text'])
text_lengths = [len(sequence) for sequence in sequences]

# Calcolo della lunghezza massima
max_length = max(text_lengths)
print("Lunghezza massima dei testi:", max_length)

In [13]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from transformers import BertJapaneseTokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, Dense, Dropout, Bidirectional, LSTM
from tensorflow.keras.preprocessing.sequence import pad_sequences
from kerastuner.tuners import RandomSearch
from transformers import BertTokenizer
from kerastuner.engine.hyperparameters import HyperParameters
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
df = pd.read_csv("/kaggle/input/japanese-newspapers-20052021/japanese_news.csv", delimiter='\t')
df['text'] = df['text'].fillna('')

/tmp/ipykernel_34/3129543821.py:17: DtypeWarning: Columns (2,3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/kaggle/input/japanese-newspapers-20052021/japanese_news.csv", delimiter='\t')


Numero di valori unici nella colonna 'source': 21


In [ ]:
print(df.head())
print(df.info())
print(df.columns)

percentages = df['source'].value_counts(normalize=True) * 100

plt.figure(figsize=(10, 6))
percentages.plot(kind='bar', color='skyblue')
plt.title('Percentage distribution for Source')
plt.xlabel('Source')
plt.ylabel('Percentage')
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

source_counts = df['source'].value_counts()

plt.figure(figsize=(10, 6))
sns.barplot(x=source_counts.index, y=source_counts.values, palette="viridis")
plt.title('Source distribution')
plt.xlabel('Source')
plt.ylabel('Number of articles')
plt.xticks(rotation=45)
plt.show()


df['date'] = pd.to_datetime(df['date'], errors='coerce')

invalid_dates = df[df['date'].isnull()]

print("Rows with invalid dates:")
print(invalid_dates)

df = df.dropna(subset=['date'])

df['year'] = df['date'].dt.year

for year, year_data in df.groupby('year'):
    source_counts = year_data['source'].value_counts()

    plt.figure(figsize=(10, 6))
    sns.barplot(x=source_counts.index, y=source_counts.values, hue=source_counts.index, palette="viridis", legend=False)
    plt.title(f'Source distribution - Year {year}')
    plt.xlabel('Source')
    plt.ylabel('Number of articles')
    plt.xticks(rotation=45)
    plt.show()
    plt.tight_layout()

df['year'] = pd.to_datetime(df['date']).dt.year
articles_per_year = df['year'].value_counts().sort_index()

plt.figure(figsize=(10,6))
articles_per_year.plot(kind='line', marker='o')
plt.title('Number of articles per year')
plt.xlabel('Year')
plt.ylabel('Number of articles')
plt.xticks(rotation=45)
plt.grid(True)
plt.show()
plt.tight_layout()

articles_per_source = df['source'].value_counts()

plt.figure(figsize=(10,6))
articles_per_source.plot(kind='line', marker='o')
plt.title('Number of articles per source')
plt.xlabel('Source')
plt.ylabel('Number of articles')
plt.xticks(rotation=45)
plt.show()
plt.tight_layout()

articles_per_year_and_source = df.groupby(['year', 'source']).size().unstack(fill_value=0)

plt.figure(figsize=(18, 12))
articles_per_year_and_source.plot(kind='line', marker='o')
plt.title('Number of articles per year per source')
plt.xlabel('Year')
plt.ylabel('Number of articles')
plt.xticks(articles_per_year_and_source.index, rotation=45)
plt.grid(True)
plt.legend(title='Source', bbox_to_anchor=(1, 1))
plt.tight_layout()
plt.show()

In [ ]:
desired_sample_size = 150000
df_sample = df.sample(n=desired_sample_size, random_state=42)

X = df_sample['text']
y = df_sample['source']

num_unique_sources = df_sample['source'].nunique()
print("Numero di valori unici nella colonna 'source':", num_unique_sources)

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [5]:
tokenizer = BertTokenizer.from_pretrained("cl-tohoku/bert-base-japanese")
X_train_tok = tokenizer(X_train.tolist(), padding=True, truncation=True, max_length=115, return_tensors="tf")
X_test_tok = tokenizer(X_test.tolist(), padding=True, truncation=True, max_length=115, return_tensors="tf")

max_len = 115  # Adjust according to the distribution of text lengths
X_train_pad = pad_sequences(X_train_tok["input_ids"], maxlen=max_len, padding='post')
X_test_pad = pad_sequences(X_test_tok["input_ids"], maxlen=max_len, padding='post')


tokenizer_config.json:   0%|          | 0.00/104 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/258k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/479 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertJapaneseTokenizer'. 
The class this function is called from is 'BertTokenizer'.


In [ ]:
def build_model(hp):
    model = Sequential()
    model.add(Embedding(input_dim=len(tokenizer), output_dim=hp.Int('embedding_output_dim', min_value=50, max_value=300, step=50)))
    model.add(Conv1D(hp.Int('conv1d_filters', min_value=64, max_value=256, step=64), 5, activation='relu'))
    model.add(Bidirectional(LSTM(hp.Int('lstm_units', min_value=32, max_value=128, step=32), dropout=hp.Float('lstm_dropout', min_value=0.1, max_value=0.5, step=0.1), recurrent_dropout=hp.Float('recurrent_dropout', min_value=0.1, max_value=0.5, step=0.1))))
    model.add(Dense(hp.Int('dense_units', min_value=32, max_value=128, step=32), activation='relu'))
    model.add(Dropout(hp.Float('dropout', min_value=0.1, max_value=0.5, step=0.1)))
    model.add(Dense(len(label_encoder.classes_), activation='softmax'))

    model.compile(loss='sparse_categorical_crossentropy', optimizer=tf.optimizers.Adam(learning_rate=hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])), metrics=['accuracy'])
    return model

tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=15,  
    executions_per_trial=1,  
    directory='my_dir',
    project_name='japanese_news'
)

tuner.search(X_train_pad, y_train, epochs=5, validation_data=(X_test_pad, y_test))

best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

model = tuner.hypermodel.build(best_hps)
history = model.fit(X_train_pad, y_train, epochs=30, batch_size=64, validation_data=(X_test_pad, y_test))

loss, accuracy = model.evaluate(X_test_pad, y_test)
print("Test Accuracy:", accuracy)


Trial 1 Complete [02h 01m 02s]
val_accuracy: 0.5042666792869568

Best val_accuracy So Far: 0.5042666792869568
Total elapsed time: 02h 01m 02s

Search: Running Trial #2

Value             |Best Value So Far |Hyperparameter
300               |50                |embedding_output_dim
192               |128               |conv1d_filters
96                |96                |lstm_units
0.5               |0.2               |lstm_dropout
0.5               |0.4               |recurrent_dropout
64                |128               |dense_units
0.5               |0.2               |dropout
0.0001            |0.001             |learning_rate

Epoch 1/5
1232/3750 [========>.....................] - ETA: 16:48 - loss: 2.6775 - accuracy: 0.1352

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



3750/3750 [==============================] - 1496s 397ms/step - loss: 2.4378 - accuracy: 0.2176 - val_loss: 2.0536 - val_accuracy: 0.3409
Epoch 2/5
3750/3750 [==============================] - 1454s 388ms/step - loss: 2.0543 - accuracy: 0.3491 - val_loss: 1.9038 - val_accuracy: 0.3856
Epoch 3/5
2173/3750 [================>.............] - ETA: 9:48 - loss: 1.9280 - accuracy: 0.3899